In [3]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import psycopg2
import os
from datetime import datetime, timedelta
from dotenv import load_dotenv
import calendar

In [12]:
# Load environment variables
load_dotenv()

# Database connection function
def connect_to_redshift():
    conn = psycopg2.connect(
        host=os.getenv('DB_HOST'),
        port=os.getenv('DB_PORT'),
        dbname=os.getenv('DB_DATABASE'),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASSWORD')
    )
    return conn

# Function to execute query and return dataframe
def execute_query(query):
    conn = connect_to_redshift()
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

In [ ]:
# Function to get data based on filters
def get_filtered_data(region, grade_level, time_period, assignment_round, tutor_set_name):
    # Base query
    query = """
    WITH base_data AS (
      SELECT
        id as trial_id,
        meta.assessment_id as id,
        created_on,
        trial_slot,
        region,
        grade_group,
        meta
      FROM application_service_teacher.auto_mapping_trial_request
      WHERE created_on >= '{start_date}'::timestamp
    ),
    all_rounds AS (
      SELECT
        id,   trial_id, created_on, trial_slot, region, grade_group, 'assignment_round_1' AS round_name,
        meta.teacher_logs.assignment_round_1 AS round_data
      FROM base_data
      WHERE meta.teacher_logs.assignment_round_1 IS NOT NULL
      UNION ALL
      SELECT
        id,   trial_id, created_on, trial_slot, region, grade_group, 'assignment_round_2',
        meta.teacher_logs.assignment_round_2
      FROM base_data
      WHERE meta.teacher_logs.assignment_round_2 IS NOT NULL
      UNION ALL
      SELECT
        id,   trial_id, created_on, trial_slot, region, grade_group, 'assignment_round_3',
        meta.teacher_logs.assignment_round_3
      FROM base_data
      WHERE meta.teacher_logs.assignment_round_3 IS NOT NULL
      UNION ALL
      SELECT
        id,   trial_id, created_on, trial_slot, region, grade_group, 'assignment_round_4',
        meta.teacher_logs.assignment_round_4
      FROM base_data
      WHERE meta.teacher_logs.assignment_round_4 IS NOT NULL
      UNION ALL
      SELECT
        id,   trial_id, created_on, trial_slot, region, grade_group, 'assignment_round_5',
        meta.teacher_logs.assignment_round_5
      FROM base_data
      WHERE meta.teacher_logs.assignment_round_5 IS NOT NULL
    ),
    all_sets AS (
      SELECT
        id,   trial_id, created_on, trial_slot, region, grade_group, round_name, 'training_complete_tutors_with_zero_trials' as tutor_set_name ,
        round_data.training_complete_tutors_with_zero_trials AS tutor_data
      FROM all_rounds
      WHERE round_data.training_complete_tutors_with_zero_trials.can_map is null
      UNION ALL
      SELECT
        id,   trial_id, created_on, trial_slot, region, grade_group, round_name, 'training_complete_tutors_with_one_or_two_trials' as tutor_set_name,
        round_data.training_complete_tutors_with_one_or_two_trials AS tutor_data
      FROM all_rounds
      WHERE round_data.training_complete_tutors_with_one_or_two_trials.can_map is null
      UNION ALL
      SELECT
        id,   trial_id, created_on, trial_slot, region, grade_group, round_name, 'active_tutors_older_version_training' AS tutor_set_name,
        round_data.active_tutors_older_version_training AS tutor_data
      FROM all_rounds
      WHERE round_data.active_tutors_older_version_training.can_map is null
      UNION ALL
      SELECT
        id,   trial_id, created_on, trial_slot, region, grade_group, round_name, 'active_tutors_latest_version_training' as tutor_set_name,
        round_data.active_tutors_latest_version_training AS tutor_data
      FROM all_rounds
      WHERE round_data.active_tutors_latest_version_training.can_map is null
    )
    SELECT
      id AS "assessment_id",
      trial_id,
      TO_CHAR(created_on, 'YYYY-MM-DD') AS "created_date",
      TO_CHAR(created_on, 'HH24') AS "Window Time",
      (trial_slot->>'from')::bigint AS "Trial Request At",
      region AS "Trial Region",
      grade_group AS "Trial Grade",
      round_name AS "Assignment Round",
      tutor_set_name AS "Tutor Set Name",
      tutor_data.total_teachers::INT AS total_teachers,
      tutor_data.after_previously_mapped_teachers,
      tutor_data.after_teachers_already_mapped_in_current_window,
      tutor_data.after_snoozed_teacher,
      tutor_data.after_ineligible_professional_review,
      tutor_data.after_disabled_region,
      tutor_data.after_max_open_trials,
      tutor_data.after_paused_on_trial_date,
      tutor_data.after_training_version_not_completed,
      tutor_data.after_max_demo_on_trial_day,
      COALESCE(tutor_data.availability_matched, 0) + COALESCE(tutor_data.availability_mismatched,0) as total_eligible_teachers,
      tutor_data.availability_matched,
      tutor_data.availability_mismatched

FROM all_sets 
  """
    
    # Calculate date range based on time period
    today = datetime.now().date()
    yesterday = today - timedelta(days=1)
    
    if time_period == 'Current Week':
        start_date = yesterday - timedelta(days=6)
    elif time_period == 'Past 2 Weeks':
        start_date = yesterday - timedelta(days=13)
    elif time_period == 'Past 4 Weeks':
        start_date = yesterday - timedelta(days=27)
    
    # Add filters to query
    filters = []
    
    # Replace placeholder in base query
    query = query.format(start_date=start_date)
    
    # Add WHERE clause if there are filters
    if region != 'All':
        filters.append(f'"Trial Region" = \'{region}\'')
    
    if grade_level != 'All':
        filters.append(f'"Trial Grade" = \'{grade_level}\'')
    
    if assignment_round != 'All':
        filters.append(f'"Assignment Round" = \'{assignment_round}\'')
    
    if tutor_set_name != 'All':
        filters.append(f'"Tutor Set Name" = \'{tutor_set_name}\'')
    
    if filters:
        query += " WHERE " + " AND ".join(filters)
    
    query += " ORDER BY \"Trial Request At\" DESC"

    df = execute_query(query)
    return df
    
    # # Execute query and return dataframe
    # try:
    #     df = execute_query(query)
    #     return df
    # except Exception as e:
       
    #     return pd.DataFrame()


In [16]:
df = get_filtered_data('NAM', 'k-2', 'Current Week', 'All', 'All')

/var/folders/j_/jynmv3t92p924jwmxww8r1n00000gp/T/ipykernel_18540/2569170852.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [17]:
df

,assessment_id,trial_id,created_date,window time,trial request at,trial region,trial grade,assignment round,tutor set name,total_teachers,...,after_snoozed_teacher,after_ineligible_professional_review,after_disabled_region,after_max_open_trials,after_paused_on_trial_date,after_training_version_not_completed,after_max_demo_on_trial_day,total_eligible_teachers,availability_matched,availability_mismatched
0,"""c1d73468-2468-11f0-a4ff-7217a269e6bc""",1771adaa-249a-11f0-b750-5a960622db9c,2025-04-29,07,1748098800,NAM,k-2,assignment_round_1,active_tutors_latest_version_training,0,...,0,0,0,0,0,0,0,0,0,0
1,"""c1d73468-2468-11f0-a4ff-7217a269e6bc""",1771adaa-249a-11f0-b750-5a960622db9c,2025-04-29,07,1748098800,NAM,k-2,assignment_round_1,active_tutors_older_version_training,236,...,235,234,212,196,196,168,168,168,0,168
2,"""ffc14884-1fba-11f0-9e19-b643f9b7d610""",1536db34-1fe3-11f0-b0f3-be15db373a98,2025-04-23,07,1747868400,NAM,k-2,assignment_round_1,active_tutors_older_version_training,6,...,3,3,3,0,0,0,0,0,0,0
3,"""ffc14884-1fba-11f0-9e19-b643f9b7d610""",1536db34-1fe3-11f0-b0f3-be15db373a98,2025-04-23,07,1747868400,NAM,k-2,assignment_round_3,active_tutors_latest_version_training,0,...,0,0,0,0,0,0,0,0,0,0
4,"""ffc14884-1fba-11f0-9e19-b643f9b7d610""",1536db34-1fe3-11f0-b0f3-be15db373a98,2025-04-23,07,1747868400,NAM,k-2,assignment_round_3,active_tutors_older_version_training,6,...,3,3,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,"""61c40922-1f16-11f0-9ce3-563561392c72""",09960242-1f1a-11f0-9d81-cad31f0ef0e7,2025-04-22,07,1745321940,NAM,k-2,assignment_round_3,active_tutors_older_version_training,6,...,2,2,2,0,0,0,0,0,0,0
1188,"""61c40922-1f16-11f0-9ce3-563561392c72""",09960242-1f1a-11f0-9d81-cad31f0ef0e7,2025-04-22,07,1745321940,NAM,k-2,assignment_round_2,active_tutors_latest_version_training,0,...,0,0,0,0,0,0,0,0,0,0
1189,"""61c40922-1f16-11f0-9ce3-563561392c72""",09960242-1f1a-11f0-9d81-cad31f0ef0e7,2025-04-22,07,1745321940,NAM,k-2,assignment_round_5,active_tutors_latest_version_training,0,...,0,0,0,0,0,0,0,0,0,0
1190,"""61c40922-1f16-11f0-9ce3-563561392c72""",09960242-1f1a-11f0-9d81-cad31f0ef0e7,2025-04-22,07,1745321940,NAM,k-2,assignment_round_4,active_tutors_older_version_training,6,...,2,2,2,0,0,0,0,0,0,0


In [7]:
df

""


In [18]:
# Function to process data for heatmap
def process_data_for_heatmap(df):
    if df.empty:
        return pd.DataFrame()

    
    # Convert all numeric columns to int
    numeric_columns = [
        'total_teachers', 'after_previously_mapped_teachers',
        'after_teachers_already_mapped_in_current_window', 'after_snoozed_teacher',
        'after_ineligible_trial_state', 'after_ineligible_professional_review',
        'after_disabled_region', 'after_max_open_trials', 'after_paused_on_trial_date',
        'after_training_version_not_completed', 'after_max_demo_on_trial_day',
        'total_eligible_teachers', 'availability_matched', 'availability_mismatched'
    ]
    
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
            
    # Rename columns to match expected names (case-insensitive)
    column_mapping = {}
    for col in df.columns:
        if col.lower() == 'trial request at':
            column_mapping[col] = 'Trial Request At'
        elif col.lower() == 'assessment_id':
            column_mapping[col] = 'assessment_id'
        elif col.lower() == 'availability_matched':
            column_mapping[col] = 'availability_matched'
    
    # Create a copy of the dataframe with renamed columns
    df_renamed = df.copy()
    df_renamed.rename(columns=column_mapping, inplace=True)
    
    
    # Check if required columns exist after renaming
    required_columns = ['Trial Request At', 'assessment_id', 'availability_matched']
    missing_columns = [col for col in required_columns if col not in df_renamed.columns]
    
    if missing_columns:
        
        # Try to find columns by case-insensitive match
        for req_col in missing_columns:
            matches = [col for col in df_renamed.columns if col.lower() == req_col.lower()]
            if matches:
                print(f"Possible match for '{req_col}': {matches}")
        
        return pd.DataFrame()
    
    # Use the renamed dataframe for further processing
    df = df_renamed
    
    # Convert 'Trial Request At' to datetime
    df['Trial Request At'] = pd.to_datetime(df['Trial Request At'], unit='s', utc=True)
    df['Trial Request At'] = df['Trial Request At'].dt.tz_convert('Asia/Kolkata')

    
    # Extract day of week and hour
    df['Day'] = df['Trial Request At'].dt.day_name()
    df['Hour'] = df['Trial Request At'].dt.hour
    

    
    # Create comprehensive time slot mapping for all 24 hours
    hour_to_slot = {
        0: '11 PM - 1 AM',  # 12 AM falls in 11 PM - 1 AM slot
        1: '1 AM - 3 AM',
        2: '1 AM - 3 AM',
        3: '3 AM - 5 AM',
        4: '3 AM - 5 AM',
        5: '5 AM - 7 AM',
        6: '5 AM - 7 AM',
        7: '7 AM - 9 AM',
        8: '7 AM - 9 AM',
        9: '9 AM - 11 AM',
        10: '9 AM - 11 AM',
        11: '11 AM - 1 PM',
        12: '11 AM - 1 PM',
        13: '1 PM - 3 PM',
        14: '1 PM - 3 PM',
        15: '3 PM - 5 PM',
        16: '3 PM - 5 PM',
        17: '5 PM - 7 PM',
        18: '5 PM - 7 PM',
        19: '7 PM - 9 PM',
        20: '7 PM - 9 PM',
        21: '9 PM - 11 PM',
        22: '9 PM - 11 PM',
        23: '11 PM - 1 AM'
    }
    
    # Map hours to time slots - comprehensive version
    def map_hour_to_slot(hour):
        if hour is None or pd.isna(hour):
            # Default to a reasonable time slot instead of 'Other'
            print(f"WARNING: Found null hour value, defaulting to '7 AM - 9 AM'")
            return '7 AM - 9 AM'
        
        try:
            hour = int(hour)
            if hour < 0 or hour > 23:
                print(f"WARNING: Hour value {hour} out of range, defaulting to '7 AM - 9 AM'")
                return '7 AM - 9 AM'
            
            return hour_to_slot[hour]
        except (ValueError, TypeError):
            print(f"WARNING: Invalid hour value {hour}, defaulting to '7 AM - 9 AM'")
            return '7 AM - 9 AM'
    
    df['Time Slot'] = df['Hour'].apply(map_hour_to_slot)
    
    # Verify no 'Other' values in Time Slot
    other_count = (df['Time Slot'] == 'Other').sum()
    if other_count > 0:
        print(f"WARNING: {other_count} rows still have 'Other' as Time Slot!")
    
    # Check time slot mapping
    print("Time Slot mapping:")
    print(df['Time Slot'].value_counts().to_dict())
    
    # Group by day and time slot
    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    time_slots_order = [
        '7 AM - 9 AM',
        '9 AM - 11 AM',
        '11 AM - 1 PM',
        '1 PM - 3 PM',
        '3 PM - 5 PM',
        '5 PM - 7 PM',
        '7 PM - 9 PM',
        '9 PM - 11 PM',
        '11 PM - 1 AM',
        '1 AM - 3 AM',
        '3 AM - 5 AM',
        '5 AM - 7 AM'
    ]
    
    # Calculate demand (unique assessment_id count)
    demand_df = df.groupby(['Day', 'Time Slot'])['assessment_id'].nunique().reset_index()
    demand_df.rename(columns={'assessment_id': 'Demand'}, inplace=True)
    
    print("Demand calculation:")
    #debug_expander.dataframe(demand_df.head())
    
    # Check which columns have non-zero values for supply calculation
    supply_candidates = ['availability_matched']
    chosen_supply_column = 'availability_matched'
    
    for col in supply_candidates:
        if col in df.columns:
            sum_val = df[col].sum()
            print(f"Sum of {col}: {sum_val}")
            if sum_val > 0:
                chosen_supply_column = col
                break
    
    if chosen_supply_column:
        print(f"Using {chosen_supply_column} for supply calculation")
        df['supply_value'] = df[chosen_supply_column]
    else:
        print("WARNING: No suitable column found with non-zero values!")
        # Use the checkbox to determine if we should use sample data
        if not st.session_state.get('use_sample_data', False):
            print("Using total_eligible_teachers as fallback")
            df['supply_value'] = df['total_eligible_teachers']
        else:
            print("Using random values (sample data mode)")
            df['supply_value'] = np.random.randint(10, 100, size=len(df))
    
    print("Final supply values:")
    print(df['supply_value'].head(10).tolist())
    print(f"Sum of supply values: {df['supply_value'].sum()}")
    
    # Calculate supply (average of supply_value)
    supply_df = df.groupby(['Day', 'Time Slot'])['supply_value'].mean().reset_index()
    supply_df.rename(columns={'supply_value': 'Supply'}, inplace=True)
    
    print("Supply calculation:")
    #debug_expander.dataframe(supply_df.head())
    
    # Merge demand and supply
    result_df = pd.merge(demand_df, supply_df, on=['Day', 'Time Slot'], how='outer')
    
    # Fill NaN values with 0
    result_df.fillna(0, inplace=True)
    
    # Calculate ratio and percentage
    result_df['Ratio'] = result_df['Supply'] / result_df['Demand'].replace(0, np.nan)
    result_df['Percentage'] = (result_df['Supply'] - result_df['Demand']) / result_df['Demand'].replace(0, np.nan) * 100
    
    # Fill NaN values with 0 again
    result_df.fillna(0, inplace=True)
    
    # Round values
    result_df['Supply'] = result_df['Supply'].round(0).astype(int)
    result_df['Demand'] = result_df['Demand'].astype(int)
    result_df['Ratio'] = result_df['Ratio'].round(2)
    result_df['Percentage'] = result_df['Percentage'].round(0).astype(int)
    
    # Create a complete grid with all days and time slots
    grid = pd.DataFrame([(day, slot) for day in days_order for slot in time_slots_order], 
                        columns=['Day', 'Time Slot'])
    
    # Merge with the result to ensure all combinations exist
    final_df = pd.merge(grid, result_df, on=['Day', 'Time Slot'], how='left')
    final_df.fillna(0, inplace=True)
    
    # Convert to pivot table format for heatmap
    pivot_df = final_df.pivot(index='Time Slot', columns='Day',
                             values=['Supply', 'Demand', 'Ratio', 'Percentage'])
    
    # Reorder indices and columns
    pivot_df = pivot_df.reindex(index=time_slots_order)
    
    # Ensure all days are present in the correct order
    for metric in ['Supply', 'Demand', 'Ratio', 'Percentage']:
        for day in days_order:
            if (metric, day) not in pivot_df.columns:
                # Add missing columns with NaN values
                pivot_df[(metric, day)] = np.nan
    
    # Reorder columns to match days_order
    column_order = []
    for metric in ['Supply', 'Demand', 'Ratio', 'Percentage']:
        for day in days_order:
            column_order.append((metric, day))
    
    # Filter to only include columns that exist in the DataFrame
    existing_columns = [col for col in column_order if col in pivot_df.columns]
    pivot_df = pivot_df[existing_columns]
    
    return pivot_df

# Function to create heatmap
def create_heatmap(pivot_df):
    if pivot_df.empty:
        st.warning("No data available for the selected filters.")
        return
    
    # Ensure days are in correct order
    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    time_slots_order = [
        '7 AM - 9 AM',
        '9 AM - 11 AM',
        '11 AM - 1 PM',
        '1 PM - 3 PM',
        '3 PM - 5 PM',
        '5 PM - 7 PM',
        '7 PM - 9 PM',
        '9 PM - 11 PM',
        '11 PM - 1 AM',
        '1 AM - 3 AM',
        '3 AM - 5 AM',
        '5 AM - 7 AM'
    ]
    
    # Create a table-like layout
    fig = go.Figure()
    
    # Get available days and time slots
    available_days = [day for day in days_order if day in pivot_df.columns.levels[1]]
    available_time_slots = [slot for slot in time_slots_order if slot in pivot_df.index]
    
    if not available_days or not available_time_slots:
        st.warning("No data available for the selected filters.")
        return
    
    # Create cells for each day and time slot
    for i, day in enumerate(days_order):
        if day not in pivot_df.columns.levels[1]:
            continue
            
        for j, time_slot in enumerate(time_slots_order):
            if time_slot not in pivot_df.index:
                continue
                
            try:
                supply = pivot_df[('Supply', day)][time_slot]
                demand = pivot_df[('Demand', day)][time_slot]
                ratio = pivot_df[('Ratio', day)][time_slot]
                percentage = pivot_df[('Percentage', day)][time_slot]
            except:
                # Skip if data is missing
                continue
            
            # Determine cell color based on percentage
            if percentage >= 50:
                color = 'rgb(0, 128, 0)'  # Dark green
            elif percentage >= 20:
                color = 'rgb(144, 238, 144)'  # Light green
            elif percentage >= 0:
                color = 'rgb(220, 255, 220)'  # Very light green
            elif percentage >= -25:
                color = 'rgb(255, 255, 0)'  # Yellow
            elif percentage >= -50:
                color = 'rgb(255, 165, 0)'  # Orange
            elif percentage >= -75:
                color = 'rgb(255, 99, 71)'  # Tomato
            else:
                color = 'rgb(255, 0, 0)'  # Red
            
            # Create text for cell
            text = f"S: {supply} | D: {demand}<br>Ratio: {ratio}<br>{percentage}%"
            
            # Add cell as a separate trace
            fig.add_trace(
                go.Scatter(
                    x=[i],
                    y=[j],
                    mode='markers',
                    marker=dict(
                        symbol='square',
                        size=40,
                        color=color,
                        line=dict(color='white', width=1)
                    ),
                    text=text,
                    hoverinfo='text',
                    showlegend=False
                )
            )
            
            # Add text annotation
            fig.add_annotation(
                x=i,
                y=j,
                text=f"S: {supply} | D: {demand}<br>Ratio: {ratio}<br>{percentage}%",
                showarrow=False,
                font=dict(color="black", size=10),
            )
    
    # Update layout
    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=list(range(len(days_order))),
            ticktext=days_order,
            tickangle=0,
            side='top',
            showgrid=True,
            gridcolor='rgba(0,0,0,0.1)',
            range=[-0.5, len(days_order)-0.5]
        ),
        yaxis=dict(
            tickmode='array',
            tickvals=list(range(len(time_slots_order))),
            ticktext=time_slots_order,
            autorange="reversed",
            showgrid=True,
            gridcolor='rgba(0,0,0,0.1)',
            range=[-0.5, len(time_slots_order)-0.5]
        ),
        width=1000,
        height=800,
        plot_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=10, r=10, t=30, b=10),
        showlegend=False,
    )
    
    # Add grid lines as shapes
    for i in range(len(days_order) + 1):
        fig.add_shape(
            type="line",
            x0=i - 0.5,
            y0=-0.5,
            x1=i - 0.5,
            y1=len(time_slots_order) - 0.5,
            line=dict(color="rgba(0,0,0,0.3)", width=1),
            layer="below"
        )
    
    for j in range(len(time_slots_order) + 1):
        fig.add_shape(
            type="line",
            x0=-0.5,
            y0=j - 0.5,
            x1=len(days_order) - 0.5,
            y1=j - 0.5,
            line=dict(color="rgba(0,0,0,0.3)", width=1),
            layer="below"
        )
    
    return fig


In [10]:
df = get_filtered_data('NAM', 'k-2', 'Current Week', 'All', 'All')

/var/folders/j_/jynmv3t92p924jwmxww8r1n00000gp/T/ipykernel_66983/2569170852.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [12]:
df['assessment_id'].nunique()

115

In [14]:
df

,assessment_id,trial_id,created_date,window time,trial request at,trial region,trial grade,assignment round,tutor set name,total_teachers,...,after_ineligible_trial_state,after_ineligible_professional_review,after_disabled_region,after_max_open_trials,after_paused_on_trial_date,after_training_version_not_completed,after_max_demo_on_trial_day,total_eligible_teachers,availability_matched,availability_mismatched
0,"""c1d73468-2468-11f0-a4ff-7217a269e6bc""",1771adaa-249a-11f0-b750-5a960622db9c,2025-04-29,07,2025-05-24 15:00,NAM,k-2,assignment_round_1,active_tutors_latest_version_training,0,...,0,0,0,0,0,0,0,0,0,0
1,"""c1d73468-2468-11f0-a4ff-7217a269e6bc""",1771adaa-249a-11f0-b750-5a960622db9c,2025-04-29,07,2025-05-24 15:00,NAM,k-2,assignment_round_1,active_tutors_older_version_training,236,...,0,234,212,196,196,168,168,168,0,168
2,"""ffc14884-1fba-11f0-9e19-b643f9b7d610""",1536db34-1fe3-11f0-b0f3-be15db373a98,2025-04-23,07,2025-05-21 23:00,NAM,k-2,assignment_round_3,active_tutors_latest_version_training,0,...,0,0,0,0,0,0,0,0,0,0
3,"""ffc14884-1fba-11f0-9e19-b643f9b7d610""",1536db34-1fe3-11f0-b0f3-be15db373a98,2025-04-23,07,2025-05-21 23:00,NAM,k-2,assignment_round_1,active_tutors_older_version_training,6,...,3,3,3,0,0,0,0,0,0,0
4,"""ffc14884-1fba-11f0-9e19-b643f9b7d610""",1536db34-1fe3-11f0-b0f3-be15db373a98,2025-04-23,07,2025-05-21 23:00,NAM,k-2,assignment_round_3,active_tutors_older_version_training,6,...,3,3,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,"""61c40922-1f16-11f0-9ce3-563561392c72""",09960242-1f1a-11f0-9d81-cad31f0ef0e7,2025-04-22,07,2025-04-22 11:39,NAM,k-2,assignment_round_3,active_tutors_older_version_training,6,...,2,2,2,0,0,0,0,0,0,0
1188,"""61c40922-1f16-11f0-9ce3-563561392c72""",09960242-1f1a-11f0-9d81-cad31f0ef0e7,2025-04-22,07,2025-04-22 11:39,NAM,k-2,assignment_round_2,active_tutors_latest_version_training,0,...,0,0,0,0,0,0,0,0,0,0
1189,"""61c40922-1f16-11f0-9ce3-563561392c72""",09960242-1f1a-11f0-9d81-cad31f0ef0e7,2025-04-22,07,2025-04-22 11:39,NAM,k-2,assignment_round_5,active_tutors_latest_version_training,0,...,0,0,0,0,0,0,0,0,0,0
1190,"""61c40922-1f16-11f0-9ce3-563561392c72""",09960242-1f1a-11f0-9d81-cad31f0ef0e7,2025-04-22,07,2025-04-22 11:39,NAM,k-2,assignment_round_1,active_tutors_latest_version_training,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
pivot_df = process_data_for_heatmap(df)

Time Slot mapping:
{'3 AM - 5 AM': 455, '7 PM - 9 PM': 192, '5 AM - 7 AM': 122, '11 PM - 1 AM': 110, '1 AM - 3 AM': 97, '9 PM - 11 PM': 83, '11 AM - 1 PM': 50, '5 PM - 7 PM': 39, '7 AM - 9 AM': 35, '9 AM - 11 AM': 6, '1 PM - 3 PM': 3}
Demand calculation:
Sum of availability_matched: 1866
Using availability_matched for supply calculation
Final supply values:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Sum of supply values: 1866
Supply calculation:


/var/folders/j_/jynmv3t92p924jwmxww8r1n00000gp/T/ipykernel_18540/2000019952.py:54: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['Trial Request At'] = pd.to_datetime(df['Trial Request At'], unit='s', utc=True)


In [15]:
pivot_df

Supply                                                   Demand  \
Day          Monday Tuesday Wednesday Thursday Friday Saturday Sunday Monday   
Time Slot                                                                      
7 AM - 9 AM     0.0    16.0       0.0      0.0    0.0      0.0    0.0    0.0   
9 AM - 11 AM    3.0     0.0       0.0      0.0    0.0      0.0    0.0    1.0   
11 AM - 1 PM    0.0     0.0       0.0      0.0    0.0      0.0    0.0    0.0   
1 PM - 3 PM     0.0     0.0       6.0      0.0    0.0      0.0    0.0    2.0   
3 PM - 5 PM     0.0     0.0       0.0      0.0    0.0      3.0    0.0    0.0   
5 PM - 7 PM     0.0     0.0       0.0      0.0    0.0      0.0    1.0    0.0   
7 PM - 9 PM     0.0     0.0       0.0      0.0    0.0      1.0    1.0    0.0   
9 PM - 11 PM    2.0     1.0       1.0      0.0    0.0      4.0    6.0    5.0   
11 PM - 1 AM    0.0     3.0       0.0      4.0    1.0      3.0    0.0    0.0   
1 AM - 3 AM     0.0     0.0       5.0      0.0    0.0     10.0    0.0    1.0   
3 AM - 5 AM     0.0     0.0       0.0      0.0    0.0      4.0    0.0    0.0   
5 AM - 7 AM     1.0     0.0       0.0      0.0    5.0      0.0    0.0    1.0   

                                ...  Ratio                 Percentage          \
Day          Tuesday Wednesday  ... Friday Saturday Sunday     Monday Tuesday   
Time Slot                       ...                                             
7 AM - 9 AM      1.0       2.0  ...   0.00     0.00   0.00        0.0  1500.0   
9 AM - 11 AM     0.0       0.0  ...   0.00     0.00   0.00      167.0     0.0   
11 AM - 1 PM     1.0       0.0  ...   0.00     0.00   0.00        0.0  -100.0   
1 PM - 3 PM      0.0       2.0  ...   0.00     0.06   0.00      -98.0     0.0   
3 PM - 5 PM      0.0       0.0  ...   0.00     0.26   0.00        0.0     0.0   
5 PM - 7 PM      0.0       0.0  ...   0.00     0.00   0.27        0.0     0.0   
7 PM - 9 PM      0.0       1.0  ...   0.00     0.14   0.62        0.0     0.0   
9 PM - 11 PM     4.0       3.0  ...   0.11     2.11   1.88      -63.0   -77.0   
11 PM - 1 AM     7.0       5.0  ...   0.24     0.81   0.00        0.0   -55.0   
1 AM - 3 AM      1.0       1.0  ...   0.50    10.50   0.00     -100.0  -100.0   
3 AM - 5 AM      0.0       0.0  ...   0.00     4.43   0.00        0.0     0.0   
5 AM - 7 AM      0.0       0.0  ...   1.80     0.00   0.00        0.0     0.0   

                                                        
Day          Wednesday Thursday Friday Saturday Sunday  
Time Slot                                               
7 AM - 9 AM     -100.0      0.0    0.0      0.0    0.0  
9 AM - 11 AM       0.0      0.0    0.0      0.0    0.0  
11 AM - 1 PM       0.0      0.0    0.0   -100.0    0.0  
1 PM - 3 PM      186.0      0.0 -100.0    -94.0 -100.0  
3 PM - 5 PM        0.0      0.0    0.0    -74.0 -100.0  
5 PM - 7 PM        0.0   -100.0    0.0   -100.0  -73.0  
7 PM - 9 PM     -100.0   -100.0    0.0    -86.0  -38.0  
9 PM - 11 PM     -79.0    -94.0  -89.0    111.0   88.0  
11 PM - 1 AM     -92.0    -49.0  -76.0    -19.0    0.0  
1 AM - 3 AM      400.0      0.0  -50.0    950.0    0.0  
3 AM - 5 AM        0.0      0.0    0.0    343.0    0.0  
5 AM - 7 AM        0.0      0.0   80.0      0.0    0.0  

[12 rows x 28 columns]

In [ ]:

# Main app
def main():
    # Title
    st.title("Cuemath Supply-Demand Analysis Dashboard")
    
    # Filters in main area
    with st.container():
        # Create three columns for filters
        col1, col2, col3 = st.columns(3)
        
        with col1:
            # Region filter
            st.markdown("**Region**")
            region_options = ['All', 'NAM', 'APAC', 'EMEA', 'IND-SUB']
            region = st.selectbox("", region_options, index=region_options.index('NAM'), label_visibility="collapsed")
        
        with col2:
            # Grade Level filter
            st.markdown("**Grade Level**")
            grade_options = ['All', 'k-2', '3-5', '6-8', '9-10', '11-12']
            grade_level = st.selectbox("", grade_options, index=grade_options.index('k-2'), label_visibility="collapsed")
        
        with col3:
            # Time Period filter
            st.markdown("**Time Period**")
            time_period_options = ['Current Week', 'Past 2 Weeks', 'Past 4 Weeks']
            time_period = st.selectbox("", time_period_options, label_visibility="collapsed")
    
    # Create expander for additional filters
    with st.expander("Additional Filters", expanded=False):
        col1, col2 = st.columns(2)
        
        with col1:
            # Assignment Round filter
            st.markdown("**Assignment Round**")
            assignment_round_options = ['All', 'assignment_round_1', 'assignment_round_2', 'assignment_round_3', 'assignment_round_4', 'assignment_round_5']
            assignment_round = st.selectbox("", assignment_round_options, label_visibility="collapsed")
        
        with col2:
            # Tutor Set Name filter
            st.markdown("**Tutor Set Name**")
            tutor_set_options = ['All', 'training_complete_tutors_with_zero_trials', 'training_complete_tutors_with_one_or_two_trials', 'active_tutors_older_version_training', 'active_tutors_latest_version_training']
            tutor_set_name = st.selectbox("", tutor_set_options, label_visibility="collapsed")
    
    # Get data based on filters
    with st.spinner("Loading data..."):
        # Add a button to use sample data instead
        use_sample_data = st.sidebar.checkbox("Use Sample Data for Testing", value=False)
        
        # Store in session state
        st.session_state['use_sample_data'] = use_sample_data
        
        # Initialize df
        df = pd.DataFrame()
        
        if not use_sample_data:
            df = get_filtered_data(region, grade_level, time_period, assignment_round, tutor_set_name)
        
        # Check if we should use sample data
        use_sample = use_sample_data or df.empty
        if use_sample:
            if df.empty and not use_sample_data:
                st.warning("No data returned from the query. Using sample data instead.")
            
            # Create sample data
            sample_data = []
            days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
            hours = list(range(24))  # Use all 24 hours to ensure coverage of all time slots
            
            for day_idx, day in enumerate(days):
                for hour in hours:
                    # Create a datetime for this day and hour
                    dt = datetime.now().replace(hour=hour, minute=0, second=0, microsecond=0)
                    if day_idx > 0:  # Adjust day
                        dt = dt - timedelta(days=day_idx)
                    
                    # Generate random supply and demand values that create different scenarios
                    # Some with surplus, some with shortage
                    if day_idx < 3:  # First 3 days have surplus
                        supply = np.random.randint(50, 100)
                        demand = np.random.randint(10, 50)
                    elif day_idx < 5:  # Next 2 days are balanced
                        supply = np.random.randint(40, 60)
                        demand = np.random.randint(40, 60)
                    else:  # Last 2 days have shortage
                        supply = np.random.randint(10, 40)
                        demand = np.random.randint(50, 100)
                    
                    sample_data.append({
                        'Trial Request At': dt,
                        'assessment_id': f"sample_{day}_{hour}",
                        'availability_matched': supply,
                        'total_eligible_teachers': supply,
                        'supply_value': supply,
                        'Trial Region': region,
                        'Trial Grade': grade_level
                    })
            
            df = pd.DataFrame(sample_data)
            st.sidebar.success("Using sample data for visualization")
    
    # Process data for heatmap
    pivot_df = process_data_for_heatmap(df)
    
    # Create and display heatmap
    if not pivot_df.empty:
        # Display heatmap title
        region_display = region if region != 'All' else 'All Regions'
        grade_display = grade_level if grade_level != 'All' else 'All Grades'
        st.markdown(f"### {region_display} Region, {grade_display} Grade: Supply-Demand Heatmap")
        
        st.markdown("Click on any cell to see detailed filter breakdown")
        
        # Display legend
        st.markdown("**Supply-Demand Gap:**")
        legend_cols = st.columns(7)
        
        with legend_cols[0]:
            st.markdown('<div style="background-color: rgb(0, 128, 0); color: white; padding: 5px; border-radius: 3px; text-align: center; font-size: 12px;">≥50% surplus</div>', unsafe_allow_html=True)
        
        with legend_cols[1]:
            st.markdown('<div style="background-color: rgb(144, 238, 144); padding: 5px; border-radius: 3px; text-align: center; font-size: 12px;">20-49% surplus</div>', unsafe_allow_html=True)
        
        with legend_cols[2]:
            st.markdown('<div style="background-color: rgb(220, 255, 220); padding: 5px; border-radius: 3px; text-align: center; font-size: 12px;">0-19% surplus</div>', unsafe_allow_html=True)
        
        with legend_cols[3]:
            st.markdown('<div style="background-color: rgb(255, 255, 0); padding: 5px; border-radius: 3px; text-align: center; font-size: 12px;">1-25% shortage</div>', unsafe_allow_html=True)
        
        with legend_cols[4]:
            st.markdown('<div style="background-color: rgb(255, 165, 0); padding: 5px; border-radius: 3px; text-align: center; font-size: 12px;">26-50% shortage</div>', unsafe_allow_html=True)
        
        with legend_cols[5]:
            st.markdown('<div style="background-color: rgb(255, 99, 71); color: white; padding: 5px; border-radius: 3px; text-align: center; font-size: 12px;">51-75% shortage</div>', unsafe_allow_html=True)
        
        with legend_cols[6]:
            st.markdown('<div style="background-color: rgb(255, 0, 0); color: white; padding: 5px; border-radius: 3px; text-align: center; font-size: 12px;">>75% shortage</div>', unsafe_allow_html=True)
        
        # Create and display heatmap
        fig = create_heatmap(pivot_df)
        st.plotly_chart(fig, use_container_width=True)
        
        # Display raw data
        with st.expander("View Raw Data"):
            st.dataframe(df)
    else:
        st.warning("No data available for the selected filters.")

if __name__ == "__main__":
    main()

In [ ]:
 # Execute query and return dataframe
    try:
        df = execute_query(query)
        
        # Debug information
        debug_expander = st.sidebar.expander("Debug Info", expanded=True)
        print({
            "Query": query,
            "Columns": df.columns.tolist(),
            "Row Count": len(df),
            "Data Types": {col: str(df[col].dtype) for col in df.columns}
        })
        
        # Show sample data
        if not df.empty:
            print("Sample Data:")
            debug_expander.dataframe(df.head(5))
        
        return df
    except Exception as e:
        st.error(f"Error executing query: {e}")
        st.error(f"Query: {query}")
        return pd.DataFrame()

# Function to process data for heatmap
def process_data_for_heatmap(df):
    if df.empty:
        return pd.DataFrame()
    
    # Create debug expander
    debug_expander = st.sidebar.expander("Data Processing Debug", expanded=True)
    print("Original columns (lowercase):")
    print(df.columns.tolist())
    
    # Convert all numeric columns to int
    numeric_columns = [
        'total_teachers', 'after_previously_mapped_teachers',
        'after_teachers_already_mapped_in_current_window', 'after_snoozed_teacher',
        'after_ineligible_trial_state', 'after_ineligible_professional_review',
        'after_disabled_region', 'after_max_open_trials', 'after_paused_on_trial_date',
        'after_training_version_not_completed', 'after_max_demo_on_trial_day',
        'total_eligible_teachers', 'availability_matched', 'availability_mismatched'
    ]
    
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
            print(f"Converted {col} to int: {df[col].dtype}")
    
    print("Data types after conversion:")
    print({col: str(df[col].dtype) for col in df.columns if col in numeric_columns})
    
    # Rename columns to match expected names (case-insensitive)
    column_mapping = {}
    for col in df.columns:
        if col.lower() == 'trial request at':
            column_mapping[col] = 'Trial Request At'
        elif col.lower() == 'assessment_id':
            column_mapping[col] = 'assessment_id'
        elif col.lower() == 'availability_matched':
            column_mapping[col] = 'availability_matched'
    
    # Create a copy of the dataframe with renamed columns
    df_renamed = df.copy()
    df_renamed.rename(columns=column_mapping, inplace=True)
    
    print("Renamed columns:")
    print(df_renamed.columns.tolist())
    
    # Check if required columns exist after renaming
    required_columns = ['Trial Request At', 'assessment_id', 'availability_matched']
    missing_columns = [col for col in required_columns if col not in df_renamed.columns]
    
    if missing_columns:
        st.error(f"Missing required columns: {', '.join(missing_columns)}")
        st.write("Available columns:", df_renamed.columns.tolist())
        st.write("Original columns:", df.columns.tolist())
        
        # Try to find columns by case-insensitive match
        for req_col in missing_columns:
            matches = [col for col in df_renamed.columns if col.lower() == req_col.lower()]
            if matches:
                st.write(f"Possible match for '{req_col}': {matches}")
        
        return pd.DataFrame()
    
    # Use the renamed dataframe for further processing
    df = df_renamed
    
    # Convert 'Trial Request At' to datetime
    df['Trial Request At'] = pd.to_datetime(df['Trial Request At'])
    print("After datetime conversion:")
    debug_expander.dataframe(df.head(3))
    
    # Extract day of week and hour
    df['Day'] = df['Trial Request At'].dt.day_name()
    df['Hour'] = df['Trial Request At'].dt.hour
    
    # Create time slots
    time_slots = {
        7: '7 AM - 9 AM',
        9: '9 AM - 11 AM',
        11: '11 AM - 1 PM',
        13: '1 PM - 3 PM',
        15: '3 PM - 5 PM',
        17: '5 PM - 7 PM',
        19: '7 PM - 9 PM',
        21: '9 PM - 11 PM',
        23: '11 PM - 1 AM',
        1: '1 AM - 3 AM',
        3: '3 AM - 5 AM',
        5: '5 AM - 7 AM'
    }
    
    # Map hours to time slots
    df['Time Slot'] = df['Hour'].apply(lambda x: time_slots.get(x - (x % 2), 'Other'))
    
    # Group by day and time slot
    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    time_slots_order = [
        '7 AM - 9 AM',
        '9 AM - 11 AM',
        '11 AM - 1 PM',
        '1 PM - 3 PM',
        '3 PM - 5 PM',
        '5 PM - 7 PM',
        '7 PM - 9 PM',
        '9 PM - 11 PM',
        '11 PM - 1 AM',
        '1 AM - 3 AM',
        '3 AM - 5 AM',
        '5 AM - 7 AM'
    ]
    
    # Calculate demand (unique assessment_id count)
    demand_df = df.groupby(['Day', 'Time Slot'])['assessment_id'].nunique().reset_index()
    demand_df.rename(columns={'assessment_id': 'Demand'}, inplace=True)